In [6]:
import os
import re
import datetime as dt
import pandas as pd
import numpy as np
import cudf
import cupy as cp
from pathlib import Path

In [7]:
start_date = dt.datetime(2024,10,15)
end_date  = dt.datetime(2024,10,31)
symbol = "BTC-USDT" #spot
exchange = "BINANCE"

In [9]:
def quality_report(
        symbol: str,
        start_date: str | dt.datetime,
        end_date: str | dt.datetime,
        data_dir: str = "./data",
        cols: tuple[str,...] = ("origin_time", "received_time"),
        data_type: str = "raw",  # 'raw' or 'normalized'
        depth: int = 20,  # max depth level for LOB data
):
    """
    Fájl-streamelt min/max + gap statisztika – max. 1 fájl a GPU-n
    
    Args:
        symbol: Kereskedési pár (pl. BTC-USDT)
        start_date: Kezdő dátum
        end_date: Végső dátum
        data_dir: Adatkönyvtár
        cols: Oszlopok listája (alapértelmezetten időbélyegek)
        data_type: 'raw' a nyers LOB adatokhoz, 'normalized' a normalizált adatokhoz
        depth: Maximum mélység a LOB adatokhoz
    """
    # ---- előkészítés -------------------------------------------------------
    sym_pat = symbol.lower().replace("-", "_")
    
    # Fájlnév minta a data_type alapján
    if data_type == "normalized":
        rex = re.compile(rf"norm_book_{sym_pat}_(\d{{8}})_(\d{{8}})\.parquet$")
    else:  # raw
        rex = re.compile(rf"book_{sym_pat}_(\d{{8}})_(\d{{8}})\.parquet$")
    
    sd = pd.to_datetime(start_date)
    ed = pd.to_datetime(end_date)

    # ---- globális statok (inkrementálisan frissítjük) ----------------------
    min_o = max_o = min_r = max_r = None
    rec_cnt = 0
    largest_gap = pd.Timedelta(0)
    largest_gap_start = None
    missing_secs = set()
    
    # Ha LOB feature-öket vizsgálunk, gyűjtsük ezeket
    if data_type in ["raw", "normalized"]:
        # LOB oszlopok generálása a depth alapján
        lob_cols = []
        for side in ['bid', 'ask']:
            for i in range(depth):
                lob_cols.append(f"{side}_{i}_price")
                lob_cols.append(f"{side}_{i}_size")
        
        # Minden oszlop, amit be akarunk olvasni
        all_cols = list(cols)
        if data_type == "raw":
            all_cols.extend(lob_cols)
        else:  # normalized esetén már minden oszlop normalizált
            all_cols.extend(lob_cols)
    else:
        all_cols = list(cols)
    
    # Minta DataFrame tárolásra a head() kiíratáshoz
    sample_df = None

    # ---- stream feldolgozás fájlról fájlra ---------------------------------
    for fn in sorted(os.listdir(data_dir)):
        m = rex.match(fn)
        if not m:
            continue
        f_sd = pd.to_datetime(m.group(1))
        f_ed = pd.to_datetime(m.group(2))
        if f_ed < sd or f_sd > ed:
            continue

        fp = os.path.join(data_dir, fn)
        print(f"Olvasás: {fp}")
        
        try:
            df = cudf.read_parquet(fp, columns=all_cols, engine="cudf")
        except Exception as e:
            print(f"Hiba a fájl olvasásakor: {e}")
            continue

        # ↓↓↓   csak a kért dátumtartomány
        if "received_time" in df.columns:
            df = df[
                (df["received_time"] >= sd) &
                (df["received_time"] <= ed)
            ]

        if df.empty:
            continue

        # Az első nem üres DataFrame-et elmentjük mintának
        if sample_df is None:
            sample_df = df.head(10).copy()

        # ------- min / max frissítés (csak ha időbélyegek vannak) ----------
        if "origin_time" in df.columns and "received_time" in df.columns:
            o_min, o_max = df["origin_time"].min(), df["origin_time"].max()
            r_min, r_max = df["received_time"].min(), df["received_time"].max()

            min_o = o_min if min_o is None else min(min_o, o_min)
            max_o = o_max if max_o is None else max(max_o, o_max)
            min_r = r_min if min_r is None else min(min_r, r_min)
            max_r = r_max if max_r is None else max(max_r, r_max)

            # ------- gaps a received_time alapján --------------------------
            recv_sorted = df["received_time"].sort_values().to_pandas()
            gaps = recv_sorted.diff().dropna()
            if not gaps.empty and gaps.max() > largest_gap:
                largest_gap = gaps.max()
                largest_gap_start = recv_sorted.iloc[gaps.argmax()]

            # ------- hiányzó másodpercek (set-union) -----------------------
            secs = recv_sorted.dt.floor("s").unique()
            missing_secs.update(
                pd.date_range(secs.min(), secs.max(), freq="s").difference(secs)
            )

        rec_cnt += len(df)

        # ------- memória felszabadítás GPU-n -------------------------------
        del df
        cp.get_default_memory_pool().free_all_blocks()

    # ---- report ------------------------------------------------------------
    print(f"\n[GPU stream report] {symbol} - {data_type.upper()} data")
    print(f"    idősáv: {sd}  →  {ed}")
    
    # Időbélyeg statisztikák (ha vannak)
    if min_r is not None and max_r is not None:
        dur_r = (max_r - min_r) / np.timedelta64(1, "s") if rec_cnt else 0
        print(f"\nOrigin_time:   {min_o}  →  {max_o}")
        print(f"Received_time: {min_r}  →  {max_r}")
        print(f"Records: {rec_cnt:,}")
        print(f"Avg rec/s: {rec_cnt/dur_r:.2f}")
        print(f"\nLargest gap: {largest_gap/np.timedelta64(1,'s'):.2f} s")
        print(f"Gap start  : {largest_gap_start}")
        print(f"\nMissing whole seconds: {len(missing_secs):,}")
    else:
        print(f"\nRecords: {rec_cnt:,}")
    
    # Minta adatok kiíratása
    if sample_df is not None:
        print("\n--- Első 10 sor minta (head) ---")
        # Pandas-ra konvertáljuk, hogy megfelelően jelenjen meg a notebook-ban
        pd_sample = sample_df.to_pandas()
        
        # Teljes megjelenítés beállítása (ne legyen truncated)
        with pd.option_context('display.max_columns', None, 'display.width', None):
            print(pd_sample)
    else:
        print("\nNincs adat a megadott paraméterekkel.")

In [ ]:
# Nyers LOB adatok ellenőrzéséhez
quality_report(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    data_dir="./data",
    data_type="raw",
)

In [10]:
# Normalizált adatok ellenőrzéséhez
quality_report(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    data_dir="./data_normalized",  # normalizált adatok mappája
    data_type="normalized",
)

Olvasás: ./data_normalized/norm_book_btc_usdt_20241001_20241015.parquet
Olvasás: ./data_normalized/norm_book_btc_usdt_20241015_20241031.parquet

[GPU stream report] BTC-USDT - NORMALIZED data
    idősáv: 2024-10-15 00:00:00  →  2024-10-31 00:00:00

Origin_time:   1970-01-01T00:00:00.000000000  →  2024-10-30T23:59:59.813999872
Received_time: 2024-10-15T00:00:00.016179456  →  2024-10-30T23:59:59.816037376
Records: 8,731,084
Avg rec/s: 6.32

Largest gap: 10.28 s
Gap start  : 2024-10-29 04:52:01.216084224

Missing whole seconds: 205

--- Első 10 sor minta (head) ---
                    origin_time                 received_time  bid_0_price  \
0 2024-10-15 00:00:00.013999872 2024-10-15 00:00:00.016179456    -1.450553   
1 2024-10-15 00:00:00.348000000 2024-10-15 00:00:01.021205248    -1.438696   
2 2024-10-15 00:00:00.513999872 2024-10-15 00:00:01.497387008    -1.438696   
3 2024-10-15 00:00:00.813999872 2024-10-15 00:00:01.909351168    -1.441234   
4 2024-10-15 00:00:00.914000128 2024-10-1

In [ ]:
def analyze_lob_features(df_book, depth=20, sample=1_000_000):
    """Basic LOB feature quality analysis"""
    import cudf
    import cupy as cp
    import re
    
    # Get LOB feature columns
    pat = r'(bid|ask)_[0-9]{1,2}_(price|size)'
    feat_cols = [c for c in df_book.columns 
                if re.match(pat, c) and int(c.split('_')[1]) < depth]
    
    # Stats collection
    rows = []
    
    for col in feat_cols:
        s = df_book[col]
        
        # Parse column info
        parts = col.split('_')
        side = parts[0]
        level = int(parts[1])
        metric = parts[2]
        
        # Calculate stats
        miss_pct = float(s.isna().mean() * 100)
        zeros_pct = float((s == 0).mean() * 100) if cudf.api.types.is_numeric_dtype(s.dtype) else None
        
        # Sample for performance if needed
        s_sample = s.sample(sample, random_state=0) if sample and len(s) > sample else s
        
        # Get basic stats
        mn = float(s.min())
        mx = float(s.max())
        mean = float(s_sample.mean())
        std = float(s_sample.std())
        
        # Quantiles
        q = s_sample.quantile([0.01, 0.25, 0.5, 0.75, 0.99])
        p01, p25, p50, p75, p99 = [float(q.iloc[i]) for i in range(5)]
        
        rows.append({
            "feature": col,
            "side": side,
            "level": level,
            "metric": metric,
            "missing_%": miss_pct,
            "zeros_%": zeros_pct,
            "min": mn,
            "p01": p01,
            "p25": p25,
            "median": p50,
            "p75": p75,
            "p99": p99,
            "max": mx,
            "mean": mean,
            "std": std,
            "cv": std/mean if mean != 0 else None
        })
        
        # GPU memory cleanup
        del s, s_sample
        cp.get_default_memory_pool().free_all_blocks()
    
    # Create report DataFrame
    report = cudf.DataFrame(rows)
    
    # Check for price issues (fixing the comparison error)
    price_issues = []
    
    # Convert to pandas for easier comparisons
    pdf = report.to_pandas()
    
    # Check bid prices (should be decreasing with level)
    bid_prices = pdf[(pdf["side"] == "bid") & (pdf["metric"] == "price")].sort_values("level")
    if len(bid_prices) > 1:
        prev_median = None
        for _, row in bid_prices.iterrows():
            if prev_median is not None and prev_median < row["median"]:
                price_issues.append(f"Bid price inversion at level {row['level']}")
            prev_median = row["median"]
    
    # Check ask prices (should be increasing with level)
    ask_prices = pdf[(pdf["side"] == "ask") & (pdf["metric"] == "price")].sort_values("level")
    if len(ask_prices) > 1:
        prev_median = None
        for _, row in ask_prices.iterrows():
            if prev_median is not None and prev_median > row["median"]:
                price_issues.append(f"Ask price inversion at level {row['level']}")
            prev_median = row["median"]
    
    # Check spread
    if not bid_prices.empty and not ask_prices.empty:
        best_bid = bid_prices[bid_prices["level"] == 0]["median"].values[0] if 0 in bid_prices["level"].values else None
        best_ask = ask_prices[ask_prices["level"] == 0]["median"].values[0] if 0 in ask_prices["level"].values else None
        
        if best_bid is not None and best_ask is not None and best_bid >= best_ask:
            price_issues.append(f"Negative/zero spread: best_bid={best_bid}, best_ask={best_ask}")
    
    # Print issues if any
    if price_issues:
        print("Price issues detected:")
        for issue in price_issues:
            print(f"- {issue}")
    
    return report

# Simple usage
def check_lob_features(df_book):
    print(f"Data shape: {df_book.shape}")
    report = analyze_lob_features(df_book)
    print(f"Features analyzed: {len(report)}")
    return report

report = check_lob_features(df_book)
print(report.to_pandas())  # For viewing